To do: Revisit state space components - order and indexing

As a final product of this notebook we wish to obtain a simulated data set given prepsecified model parameters.

In [1]:
import pickle
import numpy as np
import math

As a first step, we supply all necessary inputs for the functionality.

In [2]:
# Import the final output of pyth_create_state_space, args
# In the modular implementation pyth_create_state_space will be called by by pyth_solve
# pyth_solve is executed before pyth_simulate
file_name = "args_file.pkl"
# Open the file for reading
file_object = open(file_name,'rb')  
# load the object from the file into var args
args = pickle.load(file_object)

In [3]:
# Import the final output of pyth_backward_induction, periods_emax
# In the modular implementation pyth_create_state_space will be called by by pyth_solve
# pyth_solve is executed before pyth_simulate
file_name = "periods_emax_file.pkl"
# Open the file for reading
file_object = open(file_name,'rb')  
# load the object from the file into var args
periods_emax = pickle.load(file_object)

In [4]:
# Unpack objects from agrs
states_all, states_number_period, mapping_states_index, max_states_period = args[0], args[1], args[2], args[3]

In [5]:
# Create additional variables: (in final version, to be supplied at initialization)
num_periods = 10
num_choices = 3
educ_max = 14
educ_min = 10

num_draws_emax = 500
num_agents_sim = 10
seed_emax = 635
seed_sim = 102

# Auxiliary calculation of the education dimension (in final verion, created in attributes dictionary)
educ_range = educ_max - educ_min + 1

In [6]:
# Set constants:
# (in final version, to be supplied in a seperate module)
MISSING_INT = -99
MISSING_FLOAT = -99.00

# (in final version, to be supplied at initialization)
mu = -0.56
delta = 0.98 # discount rate

In [7]:
# Specify the values of the model parameters, optim_paras
# (in final version, to be supplied at initialization or to be estimated)
gamma_0s1 = 5.406 # coef on b0, Blundell Table VIII, p. 1733
gamma_0s2 = 5.574
gamma_0s3 = 6.949
gamma_1s1 = 0.152 # coef on gamma0, Blundell Table VIII, p. 1733
gamma_1s2 = 0.229
gamma_1s3 = 0.306
g_s1      = 0.150 #coef on g(P), Blundell Table VIII, p. 1733
g_s2      = 0.096
g_s3      = 0.116
delta_s1  = 0.081 # coef on delta, Blundell Table VIII, p. 1733
delta_s2  = 0.057
delta_s3  = 0.073
theta_p   = 0.3 # rough average of coef values in Blundell Table IX, p. 1734
theta_f   = 0.1 # rough average of coef values in Blundell Table IX, p. 1734

# Add values of the standard diviations of the disturbanceas to the vector of model parameters
# For the backward indiction, the standard diviations are the additional (here, only) parameters needed.
sigma_1 = 2.00
sigma_2 = 2.50


optim_paras = [theta_p, 
               theta_f, 
               gamma_0s1, 
               gamma_0s2, 
               gamma_0s3, 
               gamma_1s1, 
               gamma_1s2, 
               gamma_1s3, 
               delta_s1, 
               delta_s2, 
               delta_s3, 
               g_s1, 
               g_s2, 
               g_s3,
               sigma_1,
               sigma_2,]

Next, we need to genrate draws of the error term distribution. There are two types of draws which come from the same distribution. 

First, we need draws which let us numerically integrate out the error term in a Monte Carlo simulation procedure. This is necessary for computing the continuation values and, ultimately, the value functions and the model's solution. Integating out the error term represents the process in which individuals in the model form expectations about the future. Assuming rational expectations and a known error term distribution up to its parameters, individuals take the possible realization of the error terms into account by computing the expected continuatio values over the distribution of the errors. For every period we simmulate num_draws_emax draws from the error term distribution. This has been done in the backward induction procedure.

Second, we need another set of draws to represent our simulated reality. In our model, at the beginning of every new period, individuals are hit by a productivity shock. They are aware of the realization of the shock when making their labor supply choice for the period. For every period, we simmulate num_agents_sim draws of the error term distribution.

To Do: Define take_draws and transform_disturbances as functions.

In [8]:
# Create draws for simulated sample
# Create draws from the standard normal distribution
np.random.seed(seed_sim)
draws_sim_standard = np.random.multivariate_normal(np.zeros(2), np.identity(2), (num_periods, num_agents_sim))

# Transform disturbances to normal distribution with desired covariance matrix
chocks_cov = [optim_paras[14]**2, optim_paras[15]**2] #Form covariances
draws_sim_transformed = draws_sim_standard
draws_sim_transformed[:,:,0] = draws_sim_standard[:,:,0]*chocks_cov[0]
draws_sim_transformed[:,:,1] = draws_sim_standard[:,:,1]*chocks_cov[1]

#Transform by taking the exponent
draws_sim_transformed = np.exp(draws_sim_transformed)

Then, we need to define additional function called in the loop to determine agents choices. 

In [9]:
def calculate_utilities(educ_lev, exp_p, exp_f, optim_paras, draws):
    """Calculate flow utility for all choices by state and period."""
    
    # Initialize container for utilities at state space point and period
    utilities = np.tile(np.nan, num_choices)
    
    # Calculate utilities for the avaibale joices N, P, F
    
    # Non-employment
    utilities[0] = 0
    
    #Part-time employment
    utilities[1] = 18*(np.exp(np.dot(educ_lev, optim_paras[0:3])) + 
                       np.dot(educ_lev, optim_paras[3:6]) *
                       (exp_p * np.dot(educ_lev, optim_paras[6:9]) + exp_f) 
                       * (1 - np.dot(educ_lev, optim_paras[9:12])) + 1
                      + draws[0])**mu/mu + math.exp(optim_paras[12])
    
    # Full-time employment
    utilities[2] = 38*(np.exp(np.dot(educ_lev, optim_paras[0:3])) + 
                       np.dot(educ_lev, optim_paras[3:6]) *
                       (exp_p * np.dot(educ_lev, optim_paras[6:9]) + exp_f) 
                       * (1 - np.dot (educ_lev, optim_paras[9:12])) + 1
                      + draws[1])**mu/mu + math.exp(optim_paras[13])
    
    return utilities

Finally, we need to simulate a sample of initial conditions. In this example, we need to assing a value for the years of education to every agent whose life-cycle we want to simulate.

In [10]:
educ_years = list(range(educ_min, educ_max + 1))
educ_years = np.random.choice(educ_years, num_agents_sim)

Now we can simulate the model life-cycle experiences of the individuals.

In [11]:
# Start count over all simulations/row (number of agents times number of periods)
count = 0

# Initialize container for the final output
num_columns = 3 # count of the information units we wish to record
dataset = np.tile(MISSING_FLOAT, (num_agents_sim*num_periods, num_columns))

# Loop over all agents
for i in range(num_agents_sim):
    
    # Extract education information:
    educ_years_i = educ_years[i]
    
    if (educ_years_i <= 10):
        educ_lev = [1,0,0]

    elif (educ_years_i > 10) and (educ_years_i <= 12):
        educ_lev = [0,1,0]

    else:
        educ_lev = [0,0,1]
    
    # Extract the indicator of the initial state for the individual
    # depending on the individuals initial condition
    educ_years_idx = educ_years_i - educ_min
    initial_state_index = mapping_states_index[educ_years_idx, educ_years_idx, 0, 0, 0]
    
    # Assign the initial state as current state
    current_state = states_all[educ_years_idx, initial_state_index, :].copy()
    
    # Loop over all remaining
    for period in range(num_periods):
        
        # Extract state space components
        choice_lagged, exp_p, exp_f = current_state[1], current_state[2], current_state[3]
        
        # Look up the indicator for the current state
        k = mapping_states_index[period, educ_years_i - educ_min, choice_lagged - 1, exp_p, exp_f]
        
        # Record agent identifier and current period number in the dataset
        dataset[count, :2] = i, period
        
        # Calculate choice specific value functions
        # for individual, period and state space point
        
        # Extract the error term draws corresponding to
        # period number and individual
        corresponding_draws = draws_sim_transformed[period, i, :]
        
        # Calculate correspongind flow utilities
        flow_utilities = calculate_utilities(educ_lev, exp_p, exp_f, optim_paras, corresponding_draws)
        
        # Obtain continuation values for all choices
        # Initialize container for continuation values
        continuation_values = np.tile(MISSING_FLOAT, 3)
        
        if period != (num_periods - 1):
            
            # Choice: Non-employment
            # Create index for extracting the continuation value
            future_idx = mapping_states_index[period + 1, educ_years_idx, 1 - 1, exp_f, exp_p]
            # Extract continuation value
            continuation_values[0] = periods_emax[period + 1, future_idx] 

            # Choice: Part-time
            future_idx = mapping_states_index[period + 1, educ_years_idx, 2 - 1, exp_f, exp_p + 1]
            continuation_values[1] = periods_emax[period + 1, future_idx]

            # Choice: Full-time
            future_idx = mapping_states_index[period + 1, educ_years_idx, 3 - 1, exp_f + 1, exp_p]
            continuation_values[2] = periods_emax[period + 1, future_idx]
        else:
            continuation_value = np.tile(0.0, num_choices)
        
        # Calculate total values for all choices
        value_functions = flow_utilities + delta*continuation_values
        
        # Determine choice as option with highest choice specific value function
        max_idx = np.argmax(value_functions)
        
        
        # Record output
        # Record agent identifier, period number, choice, flow utility 
        dataset[count, :] = i, period, max_idx + 1
        
        
        # Update state space component experience
        current_state[max_idx + 1] += 1
        
        # Update state space component choice_lagged
        current_state[1] = max_idx + 1
        
        # Update simulation/row count
        count += 1
